In [1]:
# generate the symmetry-corrected indices for move-based convolution
import sys
import numpy as np
sys.path.append('..')

In [2]:
# test the coefficient generation logic the naive way
cell = 4

In [3]:
# display-only code, to visually check the coeffs are in the correct locations on the board
from neural.neural_ import to_pair, generate_all_moves_by_index, move_convolution_indices
from constants import BOARD_SIZE, BOARD_WIDTH, NUM_BIASES
from math import floor

all_inds, num_coeffs = move_convolution_indices()


num_coeffs -= NUM_BIASES # the first 10 in the above function are biases, don't need them
num_fields = BOARD_SIZE

cell = cell+1
tmp = all_inds[cell]
a = np.zeros([BOARD_WIDTH,BOARD_WIDTH])
for (ind, coeff) in tmp[1:]:
    pair = to_pair(ind)
    a[pair[0],pair[1]] = coeff - NUM_BIASES + 1

print(to_pair(cell))
print(a)

(1, 1)
[[ 0.  0.  0.  8.]
 [ 0.  3.  0.  0.]
 [ 0.  0.  0.  9.]
 [ 8.  0.  9.  0.]]


In [4]:
# try calling conv_stack directly from Tensorflow
import numpy as np
import tensorflow as tf

tf.reset_default_graph()

from neural.tensorflow_utils import conv_stack

with tf.Session() as sess:
    in_fields_np = np.ones([2,num_fields])
    in_fields_np[0,3] = 0
    in_fields_np[0,5] = 0
    my_pos = np.array([12, 12])
    other_pos =  np.array([5,5])
    inputs_np = np.concatenate([in_fields_np, my_pos[:,None], other_pos[:,None]],
                              1)
    #print(inputs_np.shape)
    inputs =tf.constant(inputs_np)# tf.placeholder(shape =[None, num_fields+2], dtype = tf.float32) #
    #print(sess.run(inputs))
    out = conv_stack(inputs, 5,sess)
    
    dummy = np.array([50,50])[:,None]
    #print(sess.run(get_random_index(inputs, tf.constant(dummy))))
    
    sess.run(tf.global_variables_initializer())
    stack_result = sess.run(out)#, feed_dict={inputs:inputs_np})
    print(stack_result)

[2 3 2]
[[[ 0.03371709  0.        ]
  [ 0.55621392 -0.        ]
  [-0.11715828  0.        ]]

 [[-0.36922401 -0.66139394]
  [ 1.93253136  0.54527533]
  [-1.76095617 -0.54405111]]]


In [5]:
# load game simulation data
import glob
import sys
import pickle
from data_utils import load_simulation_data

fn = '../data/states.pickle'
if True:
# try:
#     with open(fn, 'rb') as f:
#         states = pickle.load(f)
# except:
    files = glob.glob('../data/4x4tiny.pickle')
    #files = glob.glob('../data/epsgreedy/*')
    #files = glob.glob('../data/ID_x2_1000ms/result_ID*.pickle')
    #print(files)
    depths =load_simulation_data(files)
    keys = list(depths.keys())
    #print(keys)
    games = depths[keys[0]]
    #print(games[0])
    states = [state for game in games for state in game if 'score' in state] 
    print(len(states))
    with open(fn, 'wb') as f:
        pickle.dump(states,f)

12924


In [6]:
from data_utils import prepare_data_for_model
board_full, player_pos,legal_moves,next_move,y = prepare_data_for_model(states,'simple_score')
print(board_full.shape, player_pos.shape, y.shape)
y[y ==float('-inf')] = 0
y[y ==float('inf')] = 0

(12924, 16, 1) (12924, 16, 2) (12924, 1)


In [19]:
# fit the naive score as a first test of our network
from neural.keras_utils import deep_model_fun
model, Q_model = deep_model_fun(num_features = 8, num_res_modules = 2, drop_rate = 0.01, activation = 'linear')
model.summary()
model.compile(optimizer = 'adam',  loss='mean_squared_error')

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_10 (InputLayer)            (None, 16, 1)         0                                            
____________________________________________________________________________________________________
input_9 (InputLayer)             (None, 16, 2)         0                                            
____________________________________________________________________________________________________
concatenate_5 (Concatenate)      (None, 16, 3)         0           input_10[0][0]                   
                                                                   input_9[0][0]                    
____________________________________________________________________________________________________
conv1d_3 (Conv1D)                (None, 16, 8)         32          concatenate_5[0][0]     

In [ ]:
model.fit(x = [board_full, player_pos,legal_moves,next_move],y = y, batch_size = 64, epochs=10, verbose =1)
#Q_model.predict( [board_full, player_pos,legal_moves])[0]

Epoch 1/10
 6272/12924 [=============>................] - ETA: 3s - loss: 12.0246

In [9]:
states[0]

{'G': 0.9431924358580646,
 'active_player': None,
 'allscores': None,
 'depth': 10,
 'game': array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  0.,  0.]),
 'game_': None,
 'move': 8,
 'pos': (14, 15),
 'score': -inf,
 'simple_score': -5.0,
 'winner': 1.0}

In [10]:
test = [s for s,state in enumerate(states) if 'score' not in state ]
print(test)

[]


In [11]:
# Now let's get all those games where tree search actually completed
import numpy as np
from sklearn.preprocessing import OneHotEncoder
complete_states = [state for state in states]# if state['score'] == float('inf') or state['score'] == float('-inf')]
print(len(complete_states))
board_full_c, player_pos_c,legal_moves_c,next_move_c, y_c = prepare_data_for_model(complete_states,'G')
# y_c[y_c==float('inf')] = 1
# y_c[y_c==float('-inf')] = 0

12924


In [12]:
print(set(list(np.reshape(y_c,[-1]))))

{0.94319243585806456, 0.95219103750440226, 0.98029800499999997, 0.97074007470049994, 0.96137234721396003, 0.99004999999999999, 0.043241376258179587, 0.033967326046505042, 0.024504975050000044, 0.014850499999999989, 0.061239488500516082, 0.93437290638448911, 0.052330872870641809, 0.069970822679355782}


In [13]:
from neural.keras_utils import deep_model_fun
# width AND depth matter! (32, 8 seem about optimal on this dataset)
deep_model, deep_Q_model = deep_model_fun(num_features =32, num_res_modules = 8, drop_rate = 0.1, activation = 'sigmoid')
deep_model.summary()
#deep_model.compile(optimizer = 'adam',  loss='binary_crossentropy', metrics =['acc'])deep_model.compile(optimizer = 'adam',  loss='binary_crossentropy', metrics =['acc'])
deep_model.compile(optimizer = 'adam',  loss='mean_squared_error', metrics =['acc'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_6 (InputLayer)             (None, 16, 1)         0                                            
____________________________________________________________________________________________________
input_5 (InputLayer)             (None, 16, 2)         0                                            
____________________________________________________________________________________________________
concatenate_3 (Concatenate)      (None, 16, 3)         0           input_6[0][0]                    
                                                                   input_5[0][0]                    
____________________________________________________________________________________________________
conv1d_2 (Conv1D)                (None, 16, 32)        128         concatenate_3[0][0]     

In [14]:
deep_model.fit([board_full_c, player_pos_c,legal_moves_c,next_move_c], y_c, batch_size = 128, epochs=10, verbose =1, validation_split = 0.2, shuffle = True)

Train on 10339 samples, validate on 2585 samples
Epoch 1/10
10339/10339 [==============================] - 14s - loss: 0.2199 - acc: 0.0000e+00 - val_loss: 0.2216 - val_acc: 0.0000e+00
Epoch 2/10
10339/10339 [==============================] - 7s - loss: 0.1626 - acc: 0.0000e+00 - val_loss: 0.2141 - val_acc: 0.0000e+00
Epoch 3/10
10339/10339 [==============================] - 7s - loss: 0.1459 - acc: 0.0000e+00 - val_loss: 0.2077 - val_acc: 0.0000e+00
Epoch 4/10
10339/10339 [==============================] - 7s - loss: 0.1369 - acc: 0.0000e+00 - val_loss: 0.1951 - val_acc: 0.0000e+00
Epoch 5/10
10339/10339 [==============================] - 7s - loss: 0.1325 - acc: 0.0000e+00 - val_loss: 0.1687 - val_acc: 0.0000e+00
Epoch 6/10
10339/10339 [==============================] - 8s - loss: 0.1267 - acc: 0.0000e+00 - val_loss: 0.1653 - val_acc: 0.0000e+00
Epoch 7/10
10339/10339 [==============================] - 8s - loss: 0.1192 - acc: 0.0000e+00 - val_loss: 0.1490 - val_acc: 0.0000e+00
Epoch

In [15]:
deep_Q_model.save_weights('../data/deep_Q_model_weights.h5')

In [16]:
%load_ext autoreload
%autoreload 2
from collections import namedtuple
from copy import copy
from neural.neural_ import get_legal_moves
from neural.neural_agent import apply_move, get_best_move_from_model
from data_utils import prepare_data_for_model

board = np.ones(BOARD_SIZE)
#print(list(board))
board.sum()
my_pos = None
other_pos = None
game = {'pos': np.array([my_pos, other_pos]), 'game': board, 'move': 0}
game1 = apply_move(game, 0)
game2 = apply_move(game1, 1)
game3 = apply_move(game2,get_legal_moves(game2)[0])
# board, pos, _ = possible_moves_for_model(game3)
# print(board.shape, pos.shape)
get_best_move_from_model(game3, deep_Q_model)
#deep_Q_model.summary()

(8, 0.88798106)

In [17]:
from neural.neural_agent import NeuralAgent
my_agent = NeuralAgent(deep_Q_model)
my_agent.get_move(game3)


((0, 2),
 {'game': array([ 0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,
          1.,  1.,  1.]),
  'move': 8,
  'n_score': 0.88798106,
  'pos': array([1, 9])})

In [18]:
from tournament import tournament, Agent, RandomPlayer
from neural.neural_agent import NeuralAgent

my_agent = NeuralAgent(deep_Q_model)
tournament(num_matches=10, time_limit=float('inf'), 
           test_agents=[Agent(my_agent,"Neural Agent")])
a=1


This script evaluates the performance of the custom heuristic function by
comparing the strength of an agent using iterative deepening (ID) search with
alpha-beta pruning against the strength rating of agents using other heuristic
functions.  The `ID_Improved` agent provides a baseline by measuring the
performance of a basic agent using Iterative Deepening and the "improved"
heuristic (from lecture) on your hardware.  The `Student` agent then measures
the performance of Iterative Deepening and the custom heuristic against the
same opponents.


*************************
Evaluating: Neural Agent 
*************************

Playing Matches:
----------
  Match 1: Neural Agent vs   Random    	Result: 33 to 7
  Match 2: Neural Agent vs   MM_Null   	Result: 26 to 14
  Match 3: Neural Agent vs   MM_Open   	Result: 19 to 21
  Match 4: Neural Agent vs MM_Improved 	Result: 23 to 17
  Match 5: Neural Agent vs   AB_Null   	Result: 24 to 16
  Match 6: Neural Agent vs   AB_Open   	Result: 22 to 18
 